# System Identification by Lennart Ljung

In this Jupyter Notebook we will discover the ideas of System Identification. The Notebook is written by following along the book System Identification by Lennart Ljung (1st edition). Here, we will learn the knowledge needed in order to be able to apply system identification in real life. The book will be followed chronologically and Python is utilized to clarify some of the ideas.

# Chapter 1

For starters we want to develop properties between the input to a system and the output of a system. In general, for linear time-invariant and casual systems, where we assume the input is constant over a sampling period T, the output can be fully characterized by its impulse response 

$y(t) = \sum_{k=1}^{\infty}g(k)u(t-k), \quad t = 0,1,2,\dots$

Where we use t to enumerate the sampling instants. 